In [ ]:
#importing all necessary depencies
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical

1.Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file = open("/content/drive/MyDrive/shona_dataset.txt","r")
shona_text = file.read()

In [ ]:
shona_text

'\ufeffSezvo kucherechedza hunhu nekodzero yakayenzana yomunhu wese pasi pose iriyo mviromviro yomutongero uri pachokwadi norunyararo panyika.Sezvo kusatevera nekusvora kodzero dzevanhu zvakamboita kuti kuve noutsinye\nhwakarwadza pfungwa dzevanhu pasi pose, nokuuya kwenyika inevanhu vachafadzwa norusununguko nokutaura zvavanoda zvakapikirwa sezvinangwa zvinodiwa kuvanhu vese vese.Sezvo chiri chinhu chinokosha, kana munhu asingazomanikidzirwa kuita somukana wokupedzisira, kupandukira hutongi hwakaipa kana hudzvanyiriri, kuti, kodzero dzevanhu dzichengetedzwe nohutongi hwomutemo. Sezvo vanhu vese veMubatanidzwa weNyika, muchisununguko ichi, varatidzazve chitendero chavo mukodzero dzavanhu dzinokosha, uyewo muhunhu nohukoshi homunhu wega wega, munezvekodzero yakayenzana yavanhurume navanhukadzi, zvekare vazvishingisa kukurudzira budiriro mumagariro nokusimudzirwa kohupenyu huri nani murusununguko rwunokosha rwakawedzerwa. Sezvo kunzwisisa pamwechete kodzero idzi norusununguko zvichikosha

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [ ]:
# Preprocess the text
def clean_text(shona_text):
    # Convert text to lowercase
    shona_text = shona_text.lower()

    # Remove punctuation and special characters
    shona_text = re.sub(r'[^\w\s]', '', shona_text)

    # Tokenize the text
    words = word_tokenize(shona_text)

    # Remove stopwords
    nltk.download('stopwords')
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]

    # Join the words back into a cleaned text
    cleaned_text = ' '.join(words)

    return cleaned_text

cleaned_text = clean_text(shona_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
cleaned_text

'sezvo kucherechedza hunhu nekodzero yakayenzana yomunhu wese pasi pose iriyo mviromviro yomutongero uri pachokwadi norunyararo panyikasezvo kusatevera nekusvora kodzero dzevanhu zvakamboita kuti kuve noutsinye hwakarwadza pfungwa dzevanhu pasi pose nokuuya kwenyika inevanhu vachafadzwa norusununguko nokutaura zvavanoda zvakapikirwa sezvinangwa zvinodiwa kuvanhu vese vesesezvo chiri chinhu chinokosha kana munhu asingazomanikidzirwa kuita somukana wokupedzisira kupandukira hutongi hwakaipa kana hudzvanyiriri kuti kodzero dzevanhu dzichengetedzwe nohutongi hwomutemo sezvo vanhu vese vemubatanidzwa wenyika muchisununguko ichi varatidzazve chitendero chavo mukodzero dzavanhu dzinokosha uyewo muhunhu nohukoshi homunhu wega wega munezvekodzero yakayenzana yavanhurume navanhukadzi zvekare vazvishingisa kukurudzira budiriro mumagariro nokusimudzirwa kohupenyu huri nani murusununguko rwunokosha rwakawedzerwa sezvo kunzwisisa pamwechete kodzero idzi norusununguko zvichikosha zvakanyanya kuti chi

In [ ]:
# Tokenize and preprocess
tokenizer = Tokenizer()
tokenizer.fit_on_texts([cleaned_text])
total_words = len(tokenizer.word_index) + 1

from keras.preprocessing.text import text_to_word_sequence
# tokenize the document
result = text_to_word_sequence(cleaned_text)
print(result)

['sezvo', 'kucherechedza', 'hunhu', 'nekodzero', 'yakayenzana', 'yomunhu', 'wese', 'pasi', 'pose', 'iriyo', 'mviromviro', 'yomutongero', 'uri', 'pachokwadi', 'norunyararo', 'panyikasezvo', 'kusatevera', 'nekusvora', 'kodzero', 'dzevanhu', 'zvakamboita', 'kuti', 'kuve', 'noutsinye', 'hwakarwadza', 'pfungwa', 'dzevanhu', 'pasi', 'pose', 'nokuuya', 'kwenyika', 'inevanhu', 'vachafadzwa', 'norusununguko', 'nokutaura', 'zvavanoda', 'zvakapikirwa', 'sezvinangwa', 'zvinodiwa', 'kuvanhu', 'vese', 'vesesezvo', 'chiri', 'chinhu', 'chinokosha', 'kana', 'munhu', 'asingazomanikidzirwa', 'kuita', 'somukana', 'wokupedzisira', 'kupandukira', 'hutongi', 'hwakaipa', 'kana', 'hudzvanyiriri', 'kuti', 'kodzero', 'dzevanhu', 'dzichengetedzwe', 'nohutongi', 'hwomutemo', 'sezvo', 'vanhu', 'vese', 'vemubatanidzwa', 'wenyika', 'muchisununguko', 'ichi', 'varatidzazve', 'chitendero', 'chavo', 'mukodzero', 'dzavanhu', 'dzinokosha', 'uyewo', 'muhunhu', 'nohukoshi', 'homunhu', 'wega', 'wega', 'munezvekodzero', 'yakay

In [ ]:
import pickle
#saving the tokenizer
with open('tokenizer.pickle', 'wb') as handle:
  pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

2. Creating a Vocabulary

In [ ]:
#using a keras tokenizer to build an optimal vocabulary
tokenizer = Tokenizer()
tokenizer.fit_on_texts([cleaned_text])
max_vocab_size= 10000  # the vocabulary size
word_index = tokenizer.word_index
vocabulary_size = min(max_vocab_size, len(word_index))


In [ ]:
reduced_word_index = {}
for word, index in word_index.items():
    if index <= vocabulary_size:
        reduced_word_index[word] = index
    else:
        break



tokenizer.word_index = reduced_word_index
tokenizer.word_index[tokenizer.oov_token] = vocabulary_size + 1
tokenizer.num_words = vocabulary_size + 1
vocabulary_size = len(word_index)
print("Vocabulary size:", vocabulary_size)

Vocabulary size: 300


Genshim Word Embeddings

In [ ]:
from gensim.models import Word2Vec

# Train word2vec model with gensim
sentences = [sentence.split() for sentence in cleaned_text.split('.')]
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)
model.save("word2vec_shona_embeddings.model")

RNN Model 1

In [ ]:
model1 = Sequential()
model1.add(Embedding(tokenizer.num_words , 100, input_length=5))
model1.add(Bidirectional(LSTM(150, return_sequences=True)))
model1.add(Dropout(0.2))
model1.add(LSTM(100))
model1.add(Dense(tokenizer.num_words , activation='softmax'))

model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#save the trained model
model1.save('model1.h1')
#print the summary of the model
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 5, 100)            30100     
                                                                 
 bidirectional (Bidirection  (None, 5, 300)            301200    
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 5, 300)            0         
                                                                 
 lstm_1 (LSTM)               (None, 100)               160400    
                                                                 
 dense (Dense)               (None, 301)               30401     
                                                                 
Total params: 522101 (1.99 MB)
Trainable params: 522101 (1.99 MB)
Non-trainable params: 0 (0.00 Byte)
____________________

RNN Model 2 with Pretrained embeddings

In [ ]:
# Load the word2vec model
model_gensim = Word2Vec.load("word2vec_shona_embeddings.model")
embedding_matrix = np.zeros((tokenizer.num_words , 100))

model2 = Sequential()
model2.add(Embedding(tokenizer.num_words , 100, weights=[embedding_matrix], input_length=5, trainable=False))
model2.add(Bidirectional(LSTM(150, return_sequences=True)))
model2.add(Dropout(0.2))
model2.add(LSTM(100))
model2.add(Dense(tokenizer.num_words , activation='softmax'))

model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#save the trained model
model2.save('model2.h1')
#print summary of the model
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 5, 100)            30100     
                                                                 
 bidirectional_1 (Bidirecti  (None, 5, 300)            301200    
 onal)                                                           
                                                                 
 dropout_1 (Dropout)         (None, 5, 300)            0         
                                                                 
 lstm_3 (LSTM)               (None, 100)               160400    
                                                                 
 dense_1 (Dense)             (None, 301)               30401     
                                                                 
Total params: 522101 (1.99 MB)
Trainable params: 492001 (1.88 MB)
Non-trainable params: 30100 (117.58 KB)
______________

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
import pickle

# Load the model and tokenizer
model = load_model('model1.h1')
model = load_model('model2.h1')


Training Models

In [ ]:
import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [ ]:
input_sequences = []
for line in cleaned_text.split('.'):
    token_list = tokenizer.texts_to_sequences([line])[0]
for i in range(1, len(token_list)):
    n_gram_sequence = token_list[:i+1]
    input_sequences.append(n_gram_sequence)

input_sequences = np.array(pad_sequences(input_sequences, maxlen=6, padding='pre'))
X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = keras.utils.to_categorical(y, num_classes=tokenizer.num_words)

# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.4, random_state=42)



In [ ]:
# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.4, random_state=42)

In [ ]:
# Training RNN Model 1
history1 = model1.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, verbose=1)

Epoch 1/100
9/9 [==============================] - 12s 356ms/step - loss: 5.7072 - accuracy: 0.0077 - val_loss: 5.7076 - val_accuracy: 0.0114
Epoch 2/100
9/9 [==============================] - 1s 73ms/step - loss: 5.6928 - accuracy: 0.0460 - val_loss: 5.7097 - val_accuracy: 0.0343
Epoch 3/100
9/9 [==============================] - 1s 69ms/step - loss: 5.6688 - accuracy: 0.0460 - val_loss: 5.7202 - val_accuracy: 0.0629
Epoch 4/100
9/9 [==============================] - 1s 62ms/step - loss: 5.5697 - accuracy: 0.0383 - val_loss: 5.9125 - val_accuracy: 0.0629
Epoch 5/100
9/9 [==============================] - 1s 64ms/step - loss: 5.3149 - accuracy: 0.0345 - val_loss: 6.7301 - val_accuracy: 0.0629
Epoch 6/100
9/9 [==============================] - 1s 100ms/step - loss: 5.1815 - accuracy: 0.0345 - val_loss: 6.5240 - val_accuracy: 0.0629
Epoch 7/100
9/9 [==============================] - 1s 148ms/step - loss: 5.0837 - accuracy: 0.0383 - val_loss: 6.8395 - val_accuracy: 0.0686
Epoch 8/100
9/9 

In [ ]:
# Training RNN Model 2 with pretrained embeddings
history2 = model2.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, verbose=1)

Epoch 1/100
9/9 [==============================] - 7s 196ms/step - loss: 5.7073 - accuracy: 0.0153 - val_loss: 5.7072 - val_accuracy: 0.0629
Epoch 2/100
9/9 [==============================] - 0s 56ms/step - loss: 5.6972 - accuracy: 0.0345 - val_loss: 5.7084 - val_accuracy: 0.0629
Epoch 3/100
9/9 [==============================] - 1s 78ms/step - loss: 5.6689 - accuracy: 0.0345 - val_loss: 5.7260 - val_accuracy: 0.0629
Epoch 4/100
9/9 [==============================] - 1s 83ms/step - loss: 5.5249 - accuracy: 0.0345 - val_loss: 6.1004 - val_accuracy: 0.0629
Epoch 5/100
9/9 [==============================] - 1s 87ms/step - loss: 5.3392 - accuracy: 0.0345 - val_loss: 6.6919 - val_accuracy: 0.0629
Epoch 6/100
9/9 [==============================] - 1s 96ms/step - loss: 5.2675 - accuracy: 0.0345 - val_loss: 6.8173 - val_accuracy: 0.0629
Epoch 7/100
9/9 [==============================] - 1s 79ms/step - loss: 5.2310 - accuracy: 0.0345 - val_loss: 7.0198 - val_accuracy: 0.0629
Epoch 8/100
9/9 [==

Model Evaluation

In [ ]:
val_loss_model1 = history1.history['val_loss'][-1]
val_loss_model2 = history2.history['val_loss'][-1]

print(f"Validation Loss for Model 1: {val_loss_model1}")
print(f"Validation Loss for Model 2: {val_loss_model2}")

Validation Loss for Model 1: 10.128071784973145
Validation Loss for Model 2: 8.619034767150879


In [ ]:
if val_loss_model1 < val_loss_model2:
    best_model = model1
    best_model_name = "best_model1.h5"
else:
    best_model = model2
    best_model_name = "best_model2.h5"

best_model.save(best_model_name)
print(f"Saved the best model as {best_model_name}")


Saved the best model as best_model2.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the previously saved model
model = load_model('best_model2.h5')

def predict_next_words(model, tokenizer, text, num_words=1):
    """
    Predict the next set of words using the trained model.

    Args:
    - model (keras.Model): The trained model.
    - tokenizer (Tokenizer): The tokenizer object used for preprocessing.
    - text (str): The input text.
    - num_words (int): The number of words to predict.

    Returns:
    - str: The predicted words.
    """
    for _ in range(num_words):
        # Tokenize and pad the text
        sequence = tokenizer.texts_to_sequences([text])[0]
        sequence = pad_sequences([sequence], maxlen=5, padding='pre')

        # Predict the next word
        predicted_probs = model.predict(sequence, verbose=0)
        predicted = np.argmax(predicted_probs, axis=-1)

        # Convert the predicted word index to a word
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break

        # Append the predicted word to the text
        text += " " + output_word

    return ' '.join(text.split(' ')[-num_words:])


# Prompt the user for input
user_input = input("Please type five words in Shona: ")

# Predict the next words
predicted_words = predict_next_words(model, tokenizer, user_input, num_words=3)
print(f"The next words might be: {predicted_words}")


Please type five words in Shona: sezvo chiri chinhu chinokosha kana
The next words might be: kana kana kana
